# Generic Conditional Laws for Random-Fields - via:

## Universal $\mathcal{P}_1(\mathbb{R})$-Deep Neural Model $\mathcal{NN}_{1_{\mathbb{R}^n},\mathcal{D}}^{\sigma:\star}$.

---

By: [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

## What does this code do?
1. Learn Heteroskedastic Non-Linear Regression Problem
     - $Y\sim f_{\text{unkown}}(x) + \epsilon$ where $f$ is an known function and $\epsilon\sim Laplace(0,\|x\|)$
2. Learn Random Bayesian Network's Law:
    - $Y = W_J Y^{J-1}, \qquad Y^{j}\triangleq \sigma\bullet A^{j}Y^{j-1} + b^{j}, \qquad Y^0\triangleq x$

3. In the above example if $A_j = M_j\odot \tilde{A_j}$ where $\tilde{A}_j$ is a deterministic matrix and $M_j$ is a "mask", that is, a random matrix with binary entries and $\odot$ is the Hadamard product then we recover the dropout framework.

#### Mode:
Software/Hardware Testing or Real-Deal?

In [1]:
trial_run = True

### Simulation Method:

In [2]:
# Random DNN
f_unknown_mode = "Heteroskedastic_NonLinear_Regression"

# Random DNN internal noise
# f_unknown_mode = "DNN_with_Random_Weights"
Depth_Bayesian_DNN = 10
width = 20

# Random Dropout applied to trained DNN
# f_unknown_mode = "DNN_with_Bayesian_Dropout"
Dropout_rate = 0.1

# Rough SDE (time 1)
# f_unknown_mode = "Rough_SDE"

#### Rough SDE Meta-Parameters

In [3]:
# SDE with Rough Driver
N_Euler_Steps = 10**1
Hurst_Exponent = 0.01

def alpha(t,x):
    output_drift_update = t-x
    return output_drift_update

def beta(t,x):
    output_vol_update = (t+0.001)*np.diag(np.cos(x))
    return output_vol_update

---
# Training Algorithm:
---
- Random $\delta$-bounded partition on input space,
- Train deep classifier on infered classes.
---
---
---
## Notes - Why the procedure is so computationally efficient?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

## Load Auxiliaries

In [4]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())
# Load Hyper-parameter Grid
exec(open('CV_Grid.py').read())
# Load Helper Function(s)
exec(open('Helper_Functions.py').read())
# Import time separately
import time
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


# load dataset
results_path = "./outputs/models/"
results_tables_path = "./outputs/results/"
raw_data_path_folder = "./inputs/raw/"
data_path_folder = "./inputs/data/"


### Set Seed
random.seed(2021)
np.random.seed(2021)
tf.random.set_seed(2021)

Using TensorFlow backend.


Deep Feature Builder - Ready
Deep Classifier - Ready


## Meta-Parameters

### Simulation

## Problem Dimension

In [5]:
problem_dim = 3

#### Grid Hyperparameter(s)
- Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.
- Number of Training Points to Generate

In [6]:
train_test_ratio = .2
N_train_size = 10**2

Monte-Carlo Paramters

In [7]:
## Monte-Carlo
N_Monte_Carlo_Samples = 10**2

Initial radis of $\delta$-bounded random partition of $\mathcal{X}$!

In [8]:
# Hyper-parameters of Cover
delta = 0.01
Proportion_per_cluster = .5

**Note**: Setting *N_Quantizers_to_parameterize* prevents any barycenters and sub-sampling.

# Simulate from: $Y=f(X,W)$ 
- Random DNN (internal noise): 
    - $f(X,W) = f_{\text{unknown}}(X+U)$
- Random DNN: 
    - $f(X,W) = f_{\text{unknown}}(X)+W$
    
*Non-linear dependance on exhaugenous noise.*

## Heteroskedastic Regression Problem

In [9]:
if f_unknown_mode == "Heteroskedastic_NonLinear_Regression":
    #-----------#
    # Build DNN #
    #-----------#
    W_feature = np.random.uniform(size=np.array([width,problem_dim]),low=-.5,high=.5)
    W_readout = np.random.uniform(size=np.array([1,width]),low=-.5,high=.5)
    # Generate Matrices
    for i_weights in range(Depth_Bayesian_DNN):
        W_hidden_loop = np.random.uniform(size=np.array([width,width]),low=-.5,high=.5)
        if i_weights == 0:
            W_hidden_list = [W_hidden_loop]
        else:
            W_hidden_list.append(W_hidden_loop)
    # Define DNN Applier
    def f_unknown(x):
        x_internal = x.reshape(-1,)
        x_internal = np.matmul(W_feature,x)
        #Deep Layer(s)
        for i in range(Depth_Bayesian_DNN):
            W_internal = W_hidden_list[i]
            x_internal = np.matmul(W_internal,x_internal)
            x_internal = np.maximum(0,x_internal)    
        # Readout Layer
        x_internal = np.matmul(W_readout,x_internal)
        return x_internal

    # Define Simulator
    def Simulator(x_in):
        var = np.sqrt(np.sum(x_in**2))
        # Pushforward
        f_x = f_unknown(x_in)
        # Apply Noise After
        noise = np.random.laplace(0,var,N_Monte_Carlo_Samples)
        f_x_noise = np.cos(f_x) + noise
        return f_x_noise

## Bayesian DNN

In [10]:
if f_unknown_mode == "DNN_with_Random_Weights":
    def f_unknown(x):
        x_internal = x.reshape(-1,) 
        # Feature Map Layer
        W_feature = np.random.uniform(size=np.array([width,problem_dim]),low=-.5,high=.5)
        x_internal = np.matmul(W_feature,x)
    #     Deep Layer(s)
        for i in range(Depth_Bayesian_DNN):
            W_internal = np.random.uniform(size=np.array([width,width]),low=-.5,high=.5)
            x_internal = np.matmul(W_internal,x_internal)
            x_internal = np.maximum(0,x_internal)    
        # Readout Layer
        W_readout = np.random.uniform(size=np.array([1,width]),low=-.5,high=.5)
        x_internal = np.matmul(W_readout,x_internal)
        return x_internal


    def Simulator(x_in):
        for i_MC in range(N_Monte_Carlo_Samples):
            y_MC_loop = f_unknown(x_in)
            if i_MC == 0:
                y_MC = y_MC_loop
            else:
                y_MC = np.append(y_MC,y_MC_loop)
        return y_MC

## Bayesian Dropout

In [11]:
if f_unknown_mode == "DNN_with_Bayesian_Dropout":
    # Initialize Drouput Parameters
    N_Dropout = int(np.maximum(1,round(width*Dropout_rate)))
    
    #-----------#
    # Build DNN #
    #-----------#
    W_feature = np.random.uniform(size=np.array([width,problem_dim]),low=-.5,high=.5)
    W_readout = np.random.uniform(size=np.array([1,width]),low=-.5,high=.5)
    # Generate Matrices
    for i_weights in range(Depth_Bayesian_DNN):
        W_hidden_loop = np.random.uniform(size=np.array([width,width]),low=-.5,high=.5)
        if i_weights == 0:
            W_hidden_list = [W_hidden_loop]
        else:
            W_hidden_list.append(W_hidden_loop)
    # Define DNN Applier
    def f_unknown(x):
        x_internal = x.reshape(-1,)
        x_internal = np.matmul(W_feature,x)
        #Deep Layer(s)
        for i in range(Depth_Bayesian_DNN):
            W_internal = W_hidden_list[i]
            # Apply Random Dropout
            random_mask_coordinates_i = np.random.choice(range(width),N_Dropout)
            random_mask_coordinates_j = np.random.choice(range(width),N_Dropout)
            W_internal[random_mask_coordinates_i,random_mask_coordinates_j] = 0
            # Apply Dropped-out layer
            x_internal = np.matmul(W_internal,x_internal)
            x_internal = np.maximum(0,x_internal)    
        # Readout Layer
        x_internal = np.matmul(W_readout,x_internal)
        return x_internal

    def Simulator(x_in):
        for i_MC in range(N_Monte_Carlo_Samples):
            y_MC_loop = f_unknown(x_in)
            if i_MC == 0:
                y_MC = y_MC_loop
            else:
                y_MC = np.append(y_MC,y_MC_loop)
        return y_MC

### fSDEs
Lean the conditional law of $I_{X_1 \in Ball(0,1)}$ where $X_t$ solves an SDE with fBM driver.

In [12]:
if f_unknown_mode == "Rough_SDE":
    # Initialize Drouput Parameters
    N_Dropout = int(np.maximum(1,round(width*Dropout_rate)))
    
    #-----------#
    # Build DNN #
    #-----------#
    W_feature = np.random.uniform(size=np.array([width,problem_dim]),low=-.5,high=.5)
    W_readout = np.random.uniform(size=np.array([1,width]),low=-.5,high=.5)
    # Generate Matrices
    for i_weights in range(Depth_Bayesian_DNN):
        W_hidden_loop = np.random.uniform(size=np.array([width,width]),low=-.5,high=.5)
        if i_weights == 0:
            W_hidden_list = [W_hidden_loop]
        else:
            W_hidden_list.append(W_hidden_loop)
    # Define DNN Applier
    def f_unknown(x):
        x_internal = x.reshape(-1,)
        # Get fBM path
        for d in range(problem_dim):
            fBM_gen_loop = (((FBM(n=N_Euler_Steps, hurst=Hurst_Exponent, length=1, method='daviesharte')).fbm())[1:]).reshape(-1,1)
            if d == 0:
                fBM_gen = fBM_gen_loop
            else:
                fBM_gen = np.append(fBM_gen,fBM_gen_loop,axis=-1)
        # Perform Integral
        for t in range(N_Euler_Steps):
            drift_update = alpha(t/N_Euler_Steps,x_internal)/N_Euler_Steps
            vol_update = beta(t/N_Euler_Steps,x_internal)
            x_internal = x_internal + drift_update + np.matmul(vol_update,fBM_gen[t,])
        # Sum at output
        output_indicator = np.max(x_internal)
        return output_indicator

    def Simulator(x_in):
        for i_MC in range(N_Monte_Carlo_Samples):
            y_MC_loop = f_unknown(x_in)
            if i_MC == 0:
                y_MC = y_MC_loop
            else:
                y_MC = np.append(y_MC,y_MC_loop)
        return y_MC

## Initialize Data

In [13]:
N_test_size = int(np.round(N_train_size*train_test_ratio,0))

### Initialize Training Data (Inputs)

Try initial sampling-type implementation!  It worked nicely..i.e.: centers were given!

In [14]:
# Get Training Set
X_train = np.random.uniform(size=np.array([N_train_size,problem_dim]),low=.5,high=1.5)

# Get Testing Set
test_set_indices = np.random.choice(range(X_train.shape[0]),N_test_size)
X_test = X_train[test_set_indices,]
X_test = X_test + np.random.uniform(low=-(delta/np.sqrt(problem_dim)), 
                                    high = -(delta/np.sqrt(problem_dim)),
                                    size = X_test.shape)

In [15]:
# Initialize k_means
N_Quantizers_to_parameterize = int(np.maximum(2,round(Proportion_per_cluster*X_train.shape[0])))
kmeans = KMeans(n_clusters=N_Quantizers_to_parameterize, random_state=0).fit(X_train)
# Get Classes
Train_classes = np.array(pd.get_dummies(kmeans.labels_))
# Get Center Measures
Barycenters_Array_x = kmeans.cluster_centers_

### Get Barycenters
*Here we make the assumption that we can directly resample $f(X=x,U)$ if necessary...or that it is available as part of the dataset.*

In [16]:
for i in tqdm(range(Barycenters_Array_x.shape[0])):
    # Put Datum
    Bar_x_loop = Barycenters_Array_x[i,]
    # Product Monte-Carlo Sample for Input
    Bar_y_loop = (Simulator(Bar_x_loop)).reshape(1,-1)

    # Update Dataset
    if i == 0:
        Barycenters_Array = Bar_y_loop
    else:
        Barycenters_Array = np.append(Barycenters_Array,Bar_y_loop,axis=0)

100%|██████████| 50/50 [00:00<00:00, 4100.00it/s]


### Initialize Training Data (Outputs)

#### Get Training Set

In [17]:
for i in tqdm(range(X_train.shape[0])):
    # Put Datum
    x_loop = X_train[i,]
    # Product Monte-Carlo Sample for Input
    y_loop = (Simulator(x_loop)).reshape(1,-1)

    # Update Dataset
    if i == 0:
        Y_train = y_loop
        Y_train_mean_emp = np.mean(y_loop)
        Y_train_var_emp = np.mean((y_loop - np.mean(y_loop))**2)
    else:
        Y_train = np.append(Y_train,y_loop,axis=0)
        Y_train_mean_emp = np.append(Y_train_mean_emp,np.mean(y_loop))
        Y_train_var_emp = np.append(Y_train_var_emp,np.mean((y_loop - np.mean(y_loop))**2))
# Join mean and Variance Training Data
Y_train_var_emp = np.append(Y_train_mean_emp.reshape(-1,1),Y_train_var_emp.reshape(-1,1),axis=1)

100%|██████████| 100/100 [00:00<00:00, 2943.37it/s]


#### Get Test Set

In [18]:
# Start Timer
Test_Set_PredictionTime_MC = time.time()

# Generate Data
for i in tqdm(range(X_test.shape[0])):
    # Put Datum
    x_loop = X_test[i,]
    # Product Monte-Carlo Sample for Input
    y_loop = (Simulator(x_loop)).reshape(1,-1)

    # Update Dataset
    if i == 0:
        Y_test = y_loop
    else:
        Y_test = np.append(Y_test,y_loop,axis=0)
        
# End Timer
Test_Set_PredictionTime_MC = time.time() - Test_Set_PredictionTime_MC

100%|██████████| 20/20 [00:00<00:00, 5061.00it/s]


# Train Model

#### Start Timer

In [19]:
# Start Timer
Type_A_timer_Begin = time.time()

### Train Deep Classifier

In this step, we train a deep (feed-forward) classifier:
$$
\hat{f}\triangleq \operatorname{Softmax}_N\circ W_J\circ \sigma \bullet \dots \sigma \bullet W_1,
$$
to identify which barycenter we are closest to.

#### Train Deep Classifier

Re-Load Packages and CV Grid

In [20]:
# Re-Load Hyper-parameter Grid
exec(open('CV_Grid.py').read())
# Re-Load Classifier Function(s)
exec(open('Helper_Functions.py').read())

Deep Feature Builder - Ready
Deep Classifier - Ready


Train Deep Classifier

In [21]:
print("==========================================")
print("Training Classifer Portion of Type-A Model")
print("==========================================")

# Redefine (Dimension-related) Elements of Grid
param_grid_Deep_Classifier['input_dim'] = [problem_dim]
param_grid_Deep_Classifier['output_dim'] = [N_Quantizers_to_parameterize]

# Train simple deep classifier
predicted_classes_train, predicted_classes_test, N_params_deep_classifier, timer_output = build_simple_deep_classifier(n_folds = CV_folds, 
                                                                                                        n_jobs = n_jobs, 
                                                                                                        n_iter = n_iter, 
                                                                                                        param_grid_in=param_grid_Deep_Classifier, 
                                                                                                        X_train = X_train, 
                                                                                                        y_train = Train_classes,
                                                                                                        X_test = X_test)

print("===============================================")
print("Training Classifer Portion of Type Model: Done!")
print("===============================================")

Training Classifer Portion of Type-A Model
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.1s finished


Epoch 1/50
4/4 [==============================] - 0s 2ms/step - loss: 3.8968 - accuracy: 0.0100
Epoch 2/50
4/4 [==============================] - 0s 2ms/step - loss: 3.8873 - accuracy: 0.0100
Epoch 3/50
4/4 [==============================] - 0s 3ms/step - loss: 3.8790 - accuracy: 0.0100
Epoch 4/50
4/4 [==============================] - 0s 2ms/step - loss: 3.8716 - accuracy: 0.0100
Epoch 5/50
4/4 [==============================] - 0s 2ms/step - loss: 3.8643 - accuracy: 0.0100
Epoch 6/50
4/4 [==============================] - 0s 2ms/step - loss: 3.8573 - accuracy: 0.0200
Epoch 7/50
4/4 [==============================] - 0s 2ms/step - loss: 3.8505 - accuracy: 0.0700
Epoch 8/50
4/4 [==============================] - 0s 3ms/step - loss: 3.8434 - accuracy: 0.0700
Epoch 9/50
4/4 [==============================] - 0s 3ms/step - loss: 3.8366 - accuracy: 0.0700
Epoch 10/50
4/4 [==============================] - 0s 3ms/step - loss: 3.8299 - accuracy: 0.0600
Epoch 11/50
4/4 [======================

#### Get Predicted Quantized Distributions
- Each *row* of "Predicted_Weights" is the $\beta\in \Delta_N$.
- Each *Column* of "Barycenters_Array" denotes the $x_1,\dots,x_N$ making up the points of the corresponding empirical measures.

In [22]:
# Initialize Empirical Weights
empirical_weights = (np.ones(N_Monte_Carlo_Samples)/N_Monte_Carlo_Samples).reshape(-1,)

for i in range(N_Quantizers_to_parameterize):
    if i == 0:
        points_of_mass = Barycenters_Array[i,]
    else:
        points_of_mass = np.append(points_of_mass,Barycenters_Array[i,])

In [23]:
# Get Noisless Mean
direct_facts = np.apply_along_axis(f_unknown, 1, X_train)
direct_facts_test = np.apply_along_axis(f_unknown, 1, X_test)

#### Get Error(s)

In [24]:
# %run Evaluation.ipynb
exec(open('Evaluation.py').read())

#### Compute *Training* Error(s)

In [25]:
print("#--------------------#")
print(" Get Training Error(s)")
print("#--------------------#")
for i in tqdm(range((X_train.shape[0]))):
    for j in range(N_Quantizers_to_parameterize):
        b_loop = np.repeat(predicted_classes_train[i,j],N_Monte_Carlo_Samples)
        if j == 0:
            b = b_loop
        else:
            b = np.append(b,b_loop)
        b = b.reshape(-1,1)
        b = b
    b = np.array(b,dtype=float).reshape(-1,)
    b = b/N_Monte_Carlo_Samples
    
    # Compute Error(s)
    ## W1
    W1_loop = ot.emd2_1d(points_of_mass,
                         np.array(Y_train[i,]).reshape(-1,),
                         b,
                         empirical_weights)
    
    ## M1
    Mu_hat = np.sum(b*(points_of_mass))
    Mu_MC = np.mean(np.array(Y_train[i,]))
    if f_unknown_mode == "Heteroskedastic_NonLinear_Regression":
        Mu = direct_facts[i,]
    else:
        Mu = Mu_MC
        
    ### Error(s)
    Mean_loop = (Mu_hat-Mu)
    Mean_loop_MC = (Mu_hat-Mu_MC)
    
    ## Variance
    Var_hat = np.sum(((points_of_mass-Mu_hat)**2)*b)
    Var_MC = np.mean(np.array(Y_train[i]-Mu_MC)**2)
    if f_unknown_mode == "Heteroskedastic_NonLinear_Regression":
        Var = 2*np.sum(X_train[i,]**2)
    else:
        Var = Var_MC     
    ### Error(s)
    Var_loop = np.abs(Var_hat-Var)
    Var_loop_MC = np.abs(Var_MC-Var)
        
    # Skewness
    Skewness_hat = np.sum((((points_of_mass-Mu_hat)/Var)**3)*b)
    Skewness_MC = np.mean((np.array(Y_train[i]-Mu_MC)/Var_MC)**3)
    if f_unknown_mode == "Heteroskedastic_NonLinear_Regression":
        Skewness = 0
    else:
        Skewness = Skewness_MC
    ### Error(s)
    Skewness_loop = np.abs(Skewness_hat-Skewness)
    Skewness_loop_MC = np.abs(Skewness_MC-Skewness)
    
    # Skewness
    Ex_Kurtosis_hat = np.sum((((points_of_mass-Mu_hat)/Var)**4)*b) - 3
    Ex_Kurtosis_MC = np.mean((np.array(Y_train[i]-Mu_MC)/Var_MC)**4) - 3
    if f_unknown_mode == "Heteroskedastic_NonLinear_Regression":
        Ex_Kurtosis = 3
    else:
        Ex_Kurtosis = Ex_Kurtosis_MC
    ### Error(s)
    Ex_Kurtosis_loop = np.abs(Ex_Kurtosis-Ex_Kurtosis_hat)
    Ex_Kurtosis_loop_MC = np.abs(Ex_Kurtosis-Ex_Kurtosis_MC)
    
    
    
    # Get Higher Moments Loss
    Higher_momentserrors_loop,Higher_MC_momentserrors_loop = Higher_Moments_Loss(b,np.array(Y_train[i,]))
    Higher_Moments_Errors_loop = np.abs(Higher_momentserrors_loop-Higher_MC_momentserrors_loop)
    
    
    # Update
    if i == 0:
        W1_errors = W1_loop
        # Moments
        ## DNM
        Mean_errors =  Mean_loop
        Var_errors = Var_loop
        Skewness_errors = Skewness_loop
        Ex_Kurtosis_errors = Ex_Kurtosis_loop
        ## Monte-Carlo
        Mean_errors_MC =  Mean_loop_MC
        Var_errors_MC = Var_loop_MC
        Skewness_errors_MC = Skewness_loop_MC
        Ex_Kurtosis_errors_MC = Ex_Kurtosis_loop_MC
        # Higher Moments
        Higher_Moments_Errors = Higher_Moments_Errors_loop
        
        
    else:
        W1_errors = np.append(W1_errors,W1_loop)
        # Moments
        ## DNM
        Mean_errors =  np.append(Mean_errors,Mean_loop)
        Var_errors = np.append(Var_errors,Var_loop)
        Skewness_errors = np.append(Skewness_errors,Skewness_loop)
        Ex_Kurtosis_errors = np.append(Ex_Kurtosis_errors,Ex_Kurtosis_loop)
        ## Monte-Carlo
        Mean_errors_MC =  np.append(Mean_errors_MC,Mean_loop_MC)
        Var_errors_MC = np.append(Var_errors_MC,Var_loop_MC)
        Skewness_errors_MC = np.append(Skewness_errors_MC,Skewness_loop_MC)
        Ex_Kurtosis_errors_MC = np.append(Ex_Kurtosis_errors_MC,Ex_Kurtosis_loop_MC)
        # Higher Moments
        Higher_Moments_Errors = np.append(Higher_Moments_Errors,Higher_Moments_Errors_loop)
        
print("#-------------------------#")
print(" Get Training Error(s): END")
print("#-------------------------#")

  0%|          | 0/100 [00:00<?, ?it/s]

#--------------------#
 Get Training Error(s)
#--------------------#


100%|██████████| 100/100 [00:00<00:00, 178.41it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#


#### Compute *Testing* Errors

In [26]:
print("#----------------#")
print(" Get Test Error(s)")
print("#----------------#")
for i in tqdm(range((X_test.shape[0]))):
    for j in range(N_Quantizers_to_parameterize):
        b_loop_test = np.repeat(predicted_classes_test[i,j],N_Monte_Carlo_Samples)
        if j == 0:
            b_test = b_loop_test
        else:
            b_test = np.append(b,b_loop)
        b_test = b_test.reshape(-1,1)
    b_test = np.array(b,dtype=float).reshape(-1,)
    b_test = b/N_Monte_Carlo_Samples
    
    # Compute Error(s)
    ## W1
    W1_loop_test = ot.emd2_1d(points_of_mass,
                         np.array(Y_test[i,]).reshape(-1,),
                         b,
                         empirical_weights)
    
    ## M1
    Mu_hat_test = np.sum(b_test*(points_of_mass))
    Mu_MC_test = np.mean(np.array(Y_test[i,]))
    if f_unknown_mode == "Heteroskedastic_NonLinear_Regression":
        Mu_test = direct_facts_test[i,]
    else:
        Mu_test = Mu_MC_test
    Mu_test = direct_facts_test[i,]
    ### Error(s)
    Mean_loop_test = (Mu_hat_test-Mu_test)
    Mean_loop_MC_test = (Mu_hat_test-Mu_MC_test)
    
    ## M2
    Var_hat_test = np.sum(((points_of_mass-Mu_hat_test)**2)*b)
    Var_MC_test = np.mean(np.array(Y_test[i]-Mu_MC)**2)
    if f_unknown_mode == "Rough_SDE":
        Var_test = 2*np.sum(X_test[i,]**2)
    else:
        Var_test = Var_MC
    
    ### Error(s)
    Var_loop_test = np.abs(Var_hat_test-Var_test)
    Var_loop_MC_test = np.abs(Var_MC_test-Var_test)
    
    # Skewness
    Skewness_hat_test = np.sum((((points_of_mass-Mu_hat_test)/Var_test)**3)*b)
    Skewness_MC_test = np.mean((np.array(Y_test[i]-Mu_MC_test)/Var_MC_test)**3)
    if f_unknown_mode == "Heteroskedastic_NonLinear_Regression":
        Skewness_test = 0
    else:
        Skewness_test = Skewness_MC_test
    ### Error(s)
    Skewness_loop_test = np.abs(Skewness_hat_test-Skewness_test)
    Skewness_loop_MC_test = np.abs(Skewness_MC_test-Skewness_test)
    
    # Skewness
    Ex_Kurtosis_hat_test = np.sum((((points_of_mass-Mu_hat_test)/Var_test)**4)*b) - 3
    Ex_Kurtosis_MC_test = np.mean((np.array(Y_test[i]-Mu_MC_test)/Var_MC_test)**4) - 3
    if f_unknown_mode == "Heteroskedastic_NonLinear_Regression":
        Ex_Kurtosis_test = 3
    else:
        Ex_Kurtosis_test = Ex_Kurtosis_MC_test
    ### Error(s)
    Ex_Kurtosis_loop_test = np.abs(Ex_Kurtosis_test-Ex_Kurtosis_hat_test)
    Ex_Kurtosis_loop_MC_test = np.abs(Ex_Kurtosis_test-Ex_Kurtosis_MC_test)
    
    
    # Get Higher Moments Loss
    Higher_momentserrors_test_loop,Higher_MC_momentserrors_test_loop = Higher_Moments_Loss(b,np.array(Y_test[i,]))
    Higher_Moments_Errors_test_loop = np.abs(Higher_momentserrors_test_loop-Higher_MC_momentserrors_test_loop)
    
    # Update
    if i == 0:
        W1_errors_test = W1_loop_test
        # Moments
        ## DNM
        Mean_errors_test =  Mean_loop_test
        Var_errors_test = Var_loop_test
        Skewness_errors_test = Skewness_loop_test
        Ex_Kurtosis_errors_test = Ex_Kurtosis_loop_test
        ## Monte-Carlo
        Mean_errors_MC_test =  Mean_loop_MC_test
        Var_errors_MC_test = Var_loop_MC_test
        Skewness_errors_MC_test = Skewness_loop_MC_test
        Ex_Kurtosis_errors_MC_test = Ex_Kurtosis_loop_MC_test
        # Higher Moments
        Higher_Moments_test_Errors = Higher_Moments_Errors_test_loop
        
        
    else:
        W1_errors_test = np.append(W1_errors_test,W1_loop_test)
        # Moments
        ## DNM
        Mean_errors_test =  np.append(Mean_errors_test,Mean_loop_test)
        Var_errors_test = np.append(Var_errors_test,Var_loop_test)
        Skewness_errors_test = np.append(Skewness_errors_test,Skewness_loop_test)
        Ex_Kurtosis_errors_test = np.append(Ex_Kurtosis_errors_test,Ex_Kurtosis_loop_test)
        ## Monte-Carlo
        Mean_errors_MC_test =  np.append(Mean_errors_MC_test,Mean_loop_MC_test)
        Var_errors_MC_test = np.append(Var_errors_MC_test,Var_loop_MC_test)
        Skewness_errors_MC_test = np.append(Skewness_errors_MC_test,Skewness_loop_MC_test)
        Ex_Kurtosis_errors_MC_test = np.append(Ex_Kurtosis_errors_MC_test,Ex_Kurtosis_loop_MC_test)
        # Higher Moments
        Higher_Moments_test_Errors = np.append(Higher_Moments_test_Errors,Higher_Moments_Errors_test_loop)
        
        
print("#-------------------------#")
print(" Get Training Error(s): END")
print("#-------------------------#")

100%|██████████| 20/20 [00:00<00:00, 226.02it/s]

#----------------#
 Get Test Error(s)
#----------------#
#-------------------------#
 Get Training Error(s): END
#-------------------------#


#### Stop Timer

In [27]:
# Stop Timer
Type_A_timer_end = time.time()
# Compute Lapsed Time Needed For Training
Time_Lapse_Model_A = Type_A_timer_end - Type_A_timer_Begin

---
# Benchmarks
---

#### Run Pointmass Benchmark(s)
These benchmarks consist of subsets of $C(\mathbb{R}^d,\mathbb{R})$ which we lift to models in $C(\mathbb{R}^d,\cap_{1\leq q<\infty}\mathscr{P}_{q}(\mathbb{R}))$ via:
$$
\mathbb{R}^d \ni x \to f(x) \to \delta_{f(x)}\in \cap_{1\leq q<\infty}\mathcal{P}_{q}(\mathbb{R}).
$$

In [97]:
exec(open('CV_Grid.py').read())
# Notebook Mode:
# %run Evaluation.ipynb
# %run Benchmarks_Model_Builder_Pointmass_Based.ipynb
# Terminal Mode (Default):
exec(open('Evaluation.py').read())
exec(open('Benchmarks_Model_Builder_Pointmass_Based.py').read())

Deep Feature Builder - Ready
--------------
Training: ENET
--------------


  0%|          | 0/20 [00:00<?, ?it/s]

#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#


100%|██████████| 20/20 [00:00<00:00, 564.10it/s]
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Batch computation too fast (0.0065s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.0s finished
 30%|███       | 30/100 [00:00<00:00, 297.92it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
-----------------
Training: K-Ridge
-----------------
Fitting 2 folds for each of 1 candidates, totalling 2 fits
#------------#
 Get Error(s) 
#------------#


...

#------------#
 Get Error(s) 
#------------#


  0%|          | 0/20 [00:00<?, ?it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#


100%|██████████| 20/20 [00:00<00:00, 570.29it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
-----------------------
Computing Error Metrics
-----------------------


#### Run Gaussian Benchmark(s)

- Bencharm 1: [Gaussian Process Regressor](https://scikit-learn.org/stable/modules/gaussian_process.html)
- Benchmark 2: Deep Gaussian Networks:
These models train models which assume Gaussianity.  We may view these as models in $\mathcal{P}_2(\mathbb{R})$ via:
$$
\mathbb{R}^d \ni x \to (\hat{\mu}(x),\hat{\sigma}(x))\triangleq f(x) \in \mathbb{R}\times [0,\infty) \to \frac1{\hat{\sigma}(x)\sqrt{2\pi}}\exp\left(\frac{-(\cdot-\hat{\mu}(x))^2}{\hat{\sigma(x)}^2}\right) \in \mathcal{G}_1\subset \mathcal{P}_2(\mathbb{R});
$$
where $\mathcal{G}_1$ is the set of Gaussian measures on $\mathbb{R}$ equipped with the relative Wasserstein-1 topology.

In [92]:
%run Benchmarks_Model_Builder_Mean_Var.ipynb
# exec(open('Benchmarks_Model_Builder_Mean_Var.py').read())

Deep Feature Builder - Ready
Deep Feature Builder - Ready
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Batch computation too fast (0.1124s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.1s finished


Training Deep Gaussian Network!
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.2s finished


Epoch 1/50
4/4 [==============================] - 0s 1ms/step - loss: 3.4640 - mse: 21.7707 - mae: 3.4640 - mape: 89.5667
Epoch 2/50
4/4 [==============================] - 0s 1ms/step - loss: 3.4625 - mse: 21.7599 - mae: 3.4625 - mape: 89.4874
Epoch 3/50
4/4 [==============================] - 0s 2ms/step - loss: 3.4610 - mse: 21.7490 - mae: 3.4610 - mape: 89.4464
Epoch 4/50
4/4 [==============================] - 0s 4ms/step - loss: 3.4596 - mse: 21.7384 - mae: 3.4596 - mape: 89.3865
Epoch 5/50
4/4 [==============================] - 0s 2ms/step - loss: 3.4581 - mse: 21.7279 - mae: 3.4581 - mape: 89.3257
Epoch 6/50
4/4 [==============================] - 0s 2ms/step - loss: 3.4566 - mse: 21.7172 - mae: 3.4566 - mape: 89.2478
Epoch 7/50
4/4 [==============================] - 0s 2ms/step - loss: 3.4551 - mse: 21.7066 - mae: 3.4551 - mape: 89.2166
Epoch 8/50
4/4 [==============================] - 0s 3ms/step - loss: 3.4537 - mse: 21.6959 - mae: 3.4537 - mape: 89.1501
Epoch 9/50
4/4 [========

  0%|          | 0/100 [00:00<?, ?it/s]

#---------------------------------------#
 Get Training Errors for: Gaussian Models
#---------------------------------------#


100%|██████████| 20/20 [00:00<00:00, 1579.92it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#---------------------------------------#
 Get Testing Errors for: Gaussian Models
#---------------------------------------#
#------------------------#
 Get Testing Error(s): END
#------------------------#


# Summary of Mean-Centric Regression Models

#### Training Model Facts

-------------------------------------------------
Updating Performance Metrics Dataframe and Saved!
-------------------------------------------------
------------------------------------------------
Updated Performance Metrics Dataframe and Saved!
------------------------------------------------


In [109]:
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

               ENET     kRidge       GBRF       ffNN        GPR        DGN
W1       7.0215E+00 7.0213E+00 7.0697E+00 7.7400E+00 7.0137E+00 3.7538E+00
Mean     1.0204E+00 1.0135E+00 1.0958E+00 2.2312E-01 1.8091E-10 1.5339E-01
Var      6.2838E+00 6.2838E+00 6.2838E+00 6.2838E+00 6.9022E+00 5.7887E+00
Skewness 0.0000E+00 0.0000E+00 0.0000E+00 0.0000E+00 0.0000E+00 0.0000E+00
Ex_Kur   6.0000E+00 6.0000E+00 6.0000E+00 6.0000E+00 3.0000E+00 3.0000E+00


,ENET,kRidge,GBRF,ffNN,GPR,DGN
W1,7.0215E+00,7.0213E+00,7.0697E+00,7.7400E+00,7.0137E+00,3.7538E+00
Mean,1.0204E+00,1.0135E+00,1.0958E+00,2.2312E-01,1.8091E-10,1.5339E-01
Var,6.2838E+00,6.2838E+00,6.2838E+00,6.2838E+00,6.9022E+00,5.7887E+00
Skewness,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00
Ex_Kur,6.0000E+00,6.0000E+00,6.0000E+00,6.0000E+00,3.0000E+00,3.0000E+00


#### Testing Model Facts

In [99]:
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

               ENET     kRidge       GBRF       ffNN
W1       7.1055E+00 7.1052E+00 7.0163E+00 7.7363E+00
Mean     1.0201E+00 1.0133E+00 1.0201E+00 2.2610E-01
Var      6.9022E+00 6.9022E+00 6.9022E+00 6.9022E+00
Skewness 0.0000E+00 0.0000E+00 0.0000E+00 0.0000E+00
Ex_Kur   6.0000E+00 6.0000E+00 6.0000E+00 6.0000E+00


,ENET,kRidge,GBRF,ffNN
W1,7.1055E+00,7.1052E+00,7.0163E+00,7.7363E+00
Mean,1.0201E+00,1.0133E+00,1.0201E+00,2.2610E-01
Var,6.9022E+00,6.9022E+00,6.9022E+00,6.9022E+00
Skewness,0.0000E+00,0.0000E+00,0.0000E+00,0.0000E+00
Ex_Kur,6.0000E+00,6.0000E+00,6.0000E+00,6.0000E+00


#### Model Complexitie(s)

In [100]:
print(Summary_Complexity_models)
Summary_Complexity_models

        N_Params     T_Time  T_Test/T_test-MC
ENET           6 4.6286E+00        5.5304E-03
GBRF       67602 6.6005E-01        9.3637E-02
kRidge        12 3.6193E-01        4.2487E-02
ffNN         101 4.8946E+00        4.2246E+00


,N_Params,T_Time,T_Test/T_test-MC
ENET,6,4.6286E+00,5.5304E-03
GBRF,67602,6.6005E-01,9.3637E-02
kRidge,12,3.6193E-01,4.2487E-02
ffNN,101,4.8946E+00,4.2246E+00


## Mean+Variance Based Model(s)

In [90]:
%run Benchmarks_Model_Builder_Mean_Var.ipynb
# exec(open('Benchmarks_Model_Builder_Mean_Var.py').read())

Deep Feature Builder - Ready
Deep Feature Builder - Ready
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Batch computation too fast (0.1285s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.4s finished


Training Deep Gaussian Network!
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.3s finished


Epoch 1/50
4/4 [==============================] - 0s 2ms/step - loss: 3.3904 - mse: 21.2414 - mae: 3.3904 - mape: 86.7245
Epoch 2/50
4/4 [==============================] - 0s 3ms/step - loss: 3.3890 - mse: 21.2315 - mae: 3.3890 - mape: 86.6528
Epoch 3/50
4/4 [==============================] - 0s 2ms/step - loss: 3.3877 - mse: 21.2213 - mae: 3.3877 - mape: 86.6140
Epoch 4/50
4/4 [==============================] - 0s 1ms/step - loss: 3.3863 - mse: 21.2114 - mae: 3.3863 - mape: 86.5587
Epoch 5/50
4/4 [==============================] - 0s 1ms/step - loss: 3.3849 - mse: 21.2015 - mae: 3.3849 - mape: 86.5023
Epoch 6/50
4/4 [==============================] - 0s 2ms/step - loss: 3.3835 - mse: 21.1914 - mae: 3.3835 - mape: 86.4307
Epoch 7/50
4/4 [==============================] - 0s 3ms/step - loss: 3.3821 - mse: 21.1813 - mae: 3.3821 - mape: 86.4002
Epoch 8/50
4/4 [==============================] - 0s 1ms/step - loss: 3.3807 - mse: 21.1711 - mae: 3.3807 - mape: 86.3381
Epoch 9/50
4/4 [========

  0%|          | 0/100 [00:00<?, ?it/s]

#---------------------------------------#
 Get Training Errors for: Gaussian Models
#---------------------------------------#


100%|██████████| 20/20 [00:00<00:00, 1802.99it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#---------------------------------------#
 Get Testing Errors for: Gaussian Models
#---------------------------------------#
#------------------------#
 Get Testing Error(s): END
#------------------------#


In [91]:
Summary_pred_Qual_models_DGN_train[:,1]

array([3.48277365, 0.2283676 , 5.70200474, 0.        , 3.        ])

---

---

---

## Get Moment Predictions

#### Write Predictions

### Training-Set Result(s): 

In [35]:
#---------------------------------------------------------------------------------------------#
W1_95 = bootstrap(W1_errors, n=1000, func=np.mean)(.95)
W1_99 = bootstrap(W1_errors, n=1000, func=np.mean)(.99)
#---------------------------------------------------------------------------------------------#
Model_Complexity = pd.DataFrame({"N_Centers":N_Quantizers_to_parameterize,
                                 "N_Q":N_Monte_Carlo_Samples,
                                 "N_Params":N_params_deep_classifier,
                                 "Training Time":Time_Lapse_Model_A,
                                 "T_Test/T_Test-MC": (timer_output/Test_Set_PredictionTime_MC),
                                 "Time Test": timer_output,
                                 "Time EM-MC": Test_Set_PredictionTime_MC},index=["Model_Complexity_metrics"])

#---------------------------------------------------------------------------------------------#
# Compute Error Statistics/Descriptors
Summary = pd.DataFrame({"W1":np.array([np.mean(np.abs(W1_errors)),np.mean(np.abs(W1_errors_test))]),
                        "M1":np.array([np.mean(np.abs(Mean_errors)),np.mean(np.abs(Mean_errors_test))]),
                        "M1_MC":np.array([np.mean(np.abs(Mean_errors_MC)),np.mean(np.abs(Mean_errors_MC_test))]),
                        "Var":np.array([np.mean(np.abs(Var_errors)),np.mean(np.abs(Var_errors_test))]),
                        "Var_MC":np.array([np.mean(np.abs(Var_errors_MC)),np.mean(np.abs(Var_errors_MC_test))]),
                        "Skew":np.array([np.mean(np.abs(Skewness_errors)),np.mean(np.abs(Skewness_errors_test))]),
                        "Skew_MC":np.array([np.mean(np.abs(Skewness_errors_MC)),np.mean(np.abs(Skewness_errors_MC_test))]),
                        "Ex_Kur":np.array([np.mean(np.abs(Ex_Kurtosis_errors)),np.mean(np.abs(Ex_Kurtosis_errors_test))]),
                        "Ex_Kur_MC":np.array([np.mean(np.abs(Ex_Kurtosis_errors_MC)),np.mean(np.abs(Ex_Kurtosis_errors_MC_test))]),
                        "High": np.array([np.mean(np.abs(Higher_Moments_Errors)),np.mean(np.abs(Higher_Moments_test_Errors))]),
                        "N_Centers":np.array((N_Quantizers_to_parameterize,N_Quantizers_to_parameterize)),
                        "N_Q":np.array((N_Monte_Carlo_Samples,N_Monte_Carlo_Samples)),
                        "N_Params":np.array((N_params_deep_classifier,N_params_deep_classifier)),
                        "Training Time":np.array((Time_Lapse_Model_A,Time_Lapse_Model_A)),
                        "T_Test/T_Test-MC":np.array(((timer_output/Test_Set_PredictionTime_MC),(timer_output/Test_Set_PredictionTime_MC))),
                        "Problem_Dimension":np.array((problem_dim,problem_dim))
                       },index=["Train","Test"])


# Write Performance Metrics to file #
#-----------------------------------#
pd.set_option('display.float_format', '{:.4E}'.format)
Model_Complexity.to_latex((results_tables_path+"Latent_Width_NSDE"+str(width)+"Problemdimension"+str(problem_dim)+"__ModelComplexities.tex"))
pd.set_option('display.float_format', '{:.4E}'.format)
Summary.to_latex((results_tables_path+"Latent_Width_NSDE"+str(width)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS.tex"))


#---------------------------------------------------------------------------------------------#
# Update User
print(Model_Complexity)
print(Summary)

                          N_Centers  N_Q  N_Params  Training Time  \
Model_Complexity_metrics         50  100      1130     7.3712E+00   

                          T_Test/T_Test-MC  Time Test  Time EM-MC  
Model_Complexity_metrics        6.4897E+00 6.5188E-02  1.0045E-02  
              W1         M1      M1_MC        Var     Var_MC       Skew  \
Train 6.1491E-01 1.0902E+00 2.3797E-01 1.4873E+00 1.3361E+00 2.2883E-02   
Test  7.3832E-01 5.6005E-02 1.0175E+00 5.0728E+00 6.2828E+00 1.2417E-02   

         Skew_MC     Ex_Kur  Ex_Kur_MC       High  N_Centers  N_Q  N_Params  \
Train 2.8985E-02 5.6647E+00 5.8400E+00 5.9955E+02         50  100      1130   
Test  3.1887E-02 5.9878E+00 5.8452E+00 7.0056E+02         50  100      1130   

       Training Time  T_Test/T_Test-MC  Problem_Dimension  
Train     7.3712E+00        6.4897E+00                  3  
Test      7.3712E+00        6.4897E+00                  3  


# Update User

## Prediction Quality

In [36]:
Summary

,W1,M1,M1_MC,Var,Var_MC,Skew,Skew_MC,Ex_Kur,Ex_Kur_MC,High,N_Centers,N_Q,N_Params,Training Time,T_Test/T_Test-MC,Problem_Dimension
Train,6.1491E-01,1.0902E+00,2.3797E-01,1.4873E+00,1.3361E+00,2.2883E-02,2.8985E-02,5.6647E+00,5.8400E+00,5.9955E+02,50,100,1130,7.3712E+00,6.4897E+00,3
Test,7.3832E-01,5.6005E-02,1.0175E+00,5.0728E+00,6.2828E+00,1.2417E-02,3.1887E-02,5.9878E+00,5.8452E+00,7.0056E+02,50,100,1130,7.3712E+00,6.4897E+00,3


# Performance wrt Mean Prediction

---

---
# Fin
---

---